### Restart the -DeBug_List_RPT_save.pkl

In [1]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize,sys
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl

In [2]:
########################     Global settings!!!
rows_per_file = 1;  Scan_end_end = 6;
purpose_i = "Full_Exp_9_3Ts_wRest"

On_HPC =  False  
Runshort  =   True                    # a long run or a quick test
Add_Rest = False


if On_HPC:
    i_bundle = int(os.environ["PBS_ARRAY_INDEX"])
else:
    i_bundle = 1; # manually specify
Scan_start = (i_bundle-1)*rows_per_file+1;    
Scan_end   = min(Scan_start + rows_per_file-1, Scan_end_end)    
purpose = f"{purpose_i}_Case_{Scan_start}_{Scan_end}"
# interpetation: Simnon suggested, with cracking activation, heat transfer
para_csv = f"Bundle_{i_bundle}.csv"  # name of the random file to get parameters


if On_HPC:                          # Run on HPC
    Path_csv = f"InputData/{purpose_i}/" 
    Path_NiallDMA = "InputData/" 
    BasicPath=os.getcwd() 
    Para_file = Path_csv +  para_csv
else:
    import sys  
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"wip/Rio_Code/Fun_P2"))
    sys.path.append(str_path_1) 
    Path_NiallDMA = os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/InputData/") # for Linux
    BasicPath =  os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
    Para_file = BasicPath+f'/Get_Random_sets2/{purpose_i}/'+para_csv
    
if not os.path.exists(BasicPath +"/"+ purpose):
    os.mkdir(BasicPath +"/"+ purpose);
# import all functions 
from Fun_P2 import * 

# Load the random file
parameter_names, combinations = load_combinations_from_csv(Para_file)


pool_no = len(combinations)
Indexs  = np.arange(Scan_start-1,Scan_end)
index_list = Indexs+1
""" para_dict_Same = {
   "Total ageing cycles":int(tot_cyc), # 
   "Ageing cycles between RPT":int(cyc_age), # 
   "Update cycles for ageing": int(cyc_age),
} """
# Get all para
Para_dict_list = []
# get all dictionaries
for combination in combinations:
    input_dict = {}
    for parameter_name,para_value in zip(parameter_names,combination ):
        input_dict[parameter_name] = para_value
    Para_dict_list.append(input_dict)
print(f"Total scan case is {len(Para_dict_list)}")

Total scan case is 1


In [3]:
# define global index and dict for all experiment data - prepare for read!
Exp_Path = [
    "Expt 1 - Si-based Degradation/",
    "Expt 2,2 - C-based Degradation 2/",
    "Expt 3 - Cathode Degradation and Li-Plating/",
    "Expt 4 - Drive Cycle Aging (Control)/",
    "Expt 5 - Standard Cycle Aging (Control)/",]
Exp_head = [
    "Expt 1",
    "Expt 2,2",
    "Expt 3",
    "Expt 4",
    "Expt 5",]
Exp_1_Cell = ["A","B","J","D","E","F","K","L","M"];
Exp_1_Temp = {
    "A":"10","B":"10","J":"10",
    "D":"25","E":"25","F":"25",
    "K":"40","L":"40","M":"40",}
Temp_Cell_Exp_1 = {
    "10":["A","B","J"],
    "25":["D","E","F"],
    "40":["K","L","M"],}
Exp_2_Cell = ["A","B","C","D","E","F"];
Exp_2_Temp = {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",}
Temp_Cell_Exp_2 = {
    "10":["A","B"],
    "25":["C","D"],
    "40":["E","F"],}
Exp_3_Cell = ["A","B","C","D","E","F","G","H","I"];
Exp_3_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25","F":"25",
    "G":"40","H":"40","I":"40"}
Temp_Cell_Exp_3 = {
    "10":["A","B","C"],
    "25":["D","E","F"],
    "40":["G","H","I"],}
Exp_4_Cell = ["A","B","C","D","E","F","G","H"];
Exp_4_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_4 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_5_Cell = ["A","B","C","D","E","F","G","H"];
Exp_5_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_5 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_All_Cell  = [Exp_1_Cell,Exp_2_Cell,Exp_3_Cell,Exp_4_Cell,Exp_5_Cell]
Exp_Temp_Cell = [Exp_1_Temp,Exp_2_Temp,Exp_3_Temp,Exp_4_Temp,Exp_5_Temp]
Temp_Cell_Exp_All = [Temp_Cell_Exp_1,Temp_Cell_Exp_2,Temp_Cell_Exp_3,Temp_Cell_Exp_4,Temp_Cell_Exp_5]
Mark_Cell_All = [
    {
    "A":"o","B":">","J":"v",
    "D":"o","E":">","F":"v",
    "K":"o","L":">","M":"v",},
    {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">","F":"v",
    "G":"o","H":">","I":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",}]
Color_Cell_All = [
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"J":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "K":[1,0,0,0.4],"L":[1,0,0,0.4],"M":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"C":[0, 0, 0,0.7],
    "E":[1,0,0,0.4],"F":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "G":[1,0,0,0.4],"H":[1,0,0,0.4],"I":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],}]
# Update 23-06-17 
#   para="Total heat transfer coefficient [W.m-2.K-1]";
#   output="Volume-averaged cell temperature [C]"
########################  Output  ########################
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    #"CCend Negative electrode roughness ratio",
    #"CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    #"CDend Negative electrode roughness ratio",
    #"CDend Total SEI on cracks thickness [m]",
    #"REend Total SEI on cracks thickness [m]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "CD Anode potential [V]",    # self defined
    "CD Cathode potential [V]",  # self defined
    "CC Time [h]",
    "CC Terminal voltage [V]",
    "CC Anode potential [V]",    # self defined
    "CC Cathode potential [V]",  # self defined
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "Throughput capacity [A.h]",
    "CDend Total lithium capacity in particles [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    #"CDend X-averaged total SEI on cracks thickness [m]",
    #"CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode stoichiometry", 
    "CDend Negative electrode stoichiometry",
    "CDsta Positive electrode stoichiometry", 
    "CDend Positive electrode stoichiometry",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
]

keys_loc_AGE = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode surface potential difference [V]",
    "CCend SEI film overpotential [V]",
    #"CCend Negative electrode roughness ratio",
    #"CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    #"CDend Negative electrode roughness ratio",
    #"CDend Total SEI on cracks thickness [m]",
    "CDend Negative electrode surface potential difference [V]",
    "CDend SEI film overpotential [V]",
    "CDend Electrolyte diffusivity [m2.s-1]",
    "CDend Electrolyte conductivity [S.m-1]",
]
keys_tim_AGE = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "CD Anode potential [V]",    # self defined
    "CD Cathode potential [V]",  # self defined
    
    "CC Time [h]",
    "CC Terminal voltage [V]",
    "CC Anode potential [V]",    # self defined
    "CC Cathode potential [V]",  # self defined
]
keys_cyc_AGE = [];
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];
keys_all_AGE = [keys_loc_AGE,keys_tim_AGE,keys_cyc_AGE];
keys_all = [keys_all_RPT,keys_all_AGE];


# Write para - 1st round:
Values_1 = []
head_keys = list(Para_dict_list[0].keys())
head_pre = [
    "Scan No","Exp No.","Y or N",
    "Error Tot%","Error SOH %","Error LLI %",
    "Error LAM NE %","Error LAM PE %",
    "Error Res %","Error ageT %","Punish",
    "Dry out",]

head_pos = [ 
    "Total ageing cycles","Ageing cycles between RPT",
    "Update cycles for ageing",
    "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"]
Values_1 .append([*head_pre,*head_keys,*head_pos])
book_name_xlsx = f'Summary_{purpose}.xlsx';
sheet_name_xlsx = 'Output'
Target  = f'/{purpose}/'
write_excel_xlsx(
    BasicPath + Target + book_name_xlsx, 
    sheet_name_xlsx, Values_1)   
Exp_pack = [
    Exp_All_Cell,Temp_Cell_Exp_All,
    Exp_Path,Exp_head,Exp_Temp_Cell,
    book_name_xlsx,]  
# remaining set-up for experiment and create folders

if not os.path.exists(BasicPath +Target+"Mats"):
    os.mkdir(BasicPath +Target +"Mats");
if not os.path.exists(BasicPath +Target+"Plots"):
    os.mkdir(BasicPath +Target+"Plots");
if not os.path.exists(BasicPath +Target+"Excel"):
    os.mkdir(BasicPath +Target+"Excel");

Successfully create a excel file


In [4]:
import pickle
Scan_i = int(Para_dict_list[0][ 'Scan No'] )
# Load the pickled data from the file
with open(BasicPath + Target+"Mats/" + str(Scan_i)+ '-DeBug_Lists.pkl', 'rb') as file:
    DeBug_Lists = pickle.load(file)
[DeBug_List_RPT,DeBug_List_AGE] = DeBug_Lists 
if not isinstance(DeBug_List_RPT,str):
    [
        Para_update_RPT, 
        Update_Cycles_RPT, 
        dict_short_RPT, 
        str_err_RPT,
    ] = DeBug_List_RPT
    print(str_err_RPT)
else:
    print(DeBug_List_RPT)
if not isinstance(DeBug_List_AGE,str):
    [
        Para_update_AGE, 
        Update_Cycles_AGE, 
        dict_short_AGE, 
        str_err_AGE,
    ] = DeBug_List_AGE
    print(str_err_AGE)
else:
    print(DeBug_List_AGE)

Fail to run RPT due to Model error or solver error for the 3th time
Empty


In [5]:
dict_short_RPT

{'Discharge capacity [A.h]': array([0.]),
 'Throughput capacity [A.h]': array([0.]),
 'Negative particle crack length [m]': array([[2.00000025e-08],
        [2.00000028e-08],
        [2.00000028e-08],
        [2.00000031e-08],
        [2.00000038e-08]]),
 'Negative electrode active material volume fraction': array([[0.74999028],
        [0.74998941],
        [0.74998943],
        [0.74998886],
        [0.74998706]]),
 'Loss of lithium due to loss of active material in negative electrode [mol]': array([0.]),
 'Positive electrode active material volume fraction': array([[0.665],
        [0.665],
        [0.665],
        [0.665],
        [0.665]]),
 'Loss of lithium due to loss of active material in positive electrode [mol]': array([0.]),
 'Total negative electrode interfacial current density variable [A.m-2]': array([[-0.07983008],
        [-0.07396604],
        [-0.04815021],
        [ 0.07486233],
        [ 0.05265602]]),
 'Total positive electrode interfacial current density variable 

In [6]:
V_max = 4.2; V_min = 2.5
exp_RPT_text = [ (
    # refill
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (20 minute period)", 
    # 0.1C cycle 
    f"Discharge at 0.1C until {V_min} V",  
    "Rest for 3 hours (20 minute period)",  
    f"Charge at 0.1C until {V_max} V",
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (20 minute period)",
    # 0.5C cycle 
    f"Discharge at 0.5C until {V_min} V (6 minute period)",  
    "Rest for 3 hours (20 minute period)",
    f"Charge at 0.5C until {V_max} V (6 minute period)",
    f"Hold at {V_max}V until C/100",
    # Update 23-11-17: add one more 0.5C cycle to increase throughput capacity
    f"Discharge at 0.5C until {V_min} V (6 minute period)",  
    "Rest for 3 hours (20 minute period)",
    f"Charge at 0.5C until {V_max} V (6 minute period)",
    f"Hold at {V_max}V until C/100",   
    "Rest for 3 hours (20 minute period)",  
    ) ] 
print(type(exp_RPT_text[0]))
print([exp_RPT_text[0][1:]]*1)

<class 'tuple'>
[('Rest for 1 hours (20 minute period)', 'Discharge at 0.1C until 2.5 V', 'Rest for 3 hours (20 minute period)', 'Charge at 0.1C until 4.2 V', 'Hold at 4.2V until C/100', 'Rest for 1 hours (20 minute period)', 'Discharge at 0.5C until 2.5 V (6 minute period)', 'Rest for 3 hours (20 minute period)', 'Charge at 0.5C until 4.2 V (6 minute period)', 'Hold at 4.2V until C/100', 'Discharge at 0.5C until 2.5 V (6 minute period)', 'Rest for 3 hours (20 minute period)', 'Charge at 0.5C until 4.2 V (6 minute period)', 'Hold at 4.2V until C/100', 'Rest for 3 hours (20 minute period)')]


In [8]:
# RPT Model
import ast,json
model_options = Para_dict_list[0]["Model option"] 
# careful: need to change str to dict 
model_options = ast.literal_eval(model_options)
mesh_list = Para_dict_list[0]["Mesh list"]
mesh_list = json.loads(mesh_list)  
Call_RPT = RioCallback()   # 

Temper_i = Para_dict_list[0]['RPT temperature']
index_exp = int(Para_dict_list[0]['Exp No.'])
V_max=4.2; V_min = 2.5
Pack_return = Initialize_exp_text( index_exp, V_max, V_min, Add_Rest)
[
    exp_AGE_text, step_AGE_CD, step_AGE_CC, step_AGE_CV,
    exp_RPT_text, exp_RPT_GITT_text, exp_adjust_before_age,
    step_0p1C_CD, step_0p1C_CC, step_0p1C_RE, step_0p5C_CD
    ]   = Pack_return
ModelExperiment    = pb.Experiment( exp_RPT_text*1 + exp_adjust_before_age*1 ) 
discharge_time_mins = 0.15* 60 * 4.86491/5

Model = pb.lithium_ion.DFN(options=model_options)
Model_new = Model.set_initial_conditions_from(dict_short_RPT, inplace=False)
submesh_types = Model.default_submesh_types
var = pb.standard_spatial_vars  
var_pts = {
    var.x_n: int(mesh_list[0]),  
    var.x_s: int(mesh_list[1]),  
    var.x_p: int(mesh_list[2]),  
    var.r_n: int(mesh_list[3]),  
    var.r_p: int(mesh_list[4]),  
    }
# compare whether further charge at 100%SOC
Exp_cha_at_100    = pb.Experiment(
    [(
        # refill
        "Rest for 10 s", 
        "Rest for 1 hours (20 minute period)", 
        # 0.1C cycle 
        f"Discharge at 0.1C until {V_min} V",  
        "Rest for 3 hours (20 minute period)",  
        f"Charge at 0.1C until {V_max} V",
        f"Hold at {V_max}V until C/100",
        "Rest for 1 hours (20 minute period)",
        # 0.5C cycle 
        f"Discharge at 0.5C until {V_min} V (6 minute period)",  
        "Rest for 3 hours (20 minute period)",
        f"Charge at 0.5C until {V_max} V (6 minute period)",
        f"Hold at {V_max}V until C/100",
        # Update 23-11-17: add one more 0.5C cycle to increase throughput capacity
        f"Discharge at 0.5C until {V_min} V (6 minute period)",  
        "Rest for 3 hours (20 minute period)",
        f"Charge at 0.5C until {V_max} V (6 minute period)",
        f"Hold at {V_max}V until C/100",
        "Rest for 3 hours (20 minute period)", 
        # refill and adjust to target SOC!!!
        f"Charge at 0.3C until {V_max}V",
        f"Hold at {V_max}V until C/100",
        "Rest for 1 hours (20 minute period)", 
        # adjust to target SOC, SOC before this step must be 100%
        f"Discharge at 1C for {discharge_time_mins} minutes or until {V_min}V", # discharge for 15%SOC
        "Rest for 3 hours (20 minute period)", 
        ),  ])  
Exp_wocha_at_100    = pb.Experiment(
    [(
        # refill
        "Rest for 10 s", 
        "Rest for 1 hours (20 minute period)", 
        # 0.1C cycle 
        f"Discharge at 0.1C until {V_min} V",  
        "Rest for 3 hours (20 minute period)",  
        f"Charge at 0.1C until {V_max} V",
        f"Hold at {V_max}V until C/100",
        "Rest for 1 hours (20 minute period)",
        # 0.5C cycle 
        f"Discharge at 0.5C until {V_min} V (6 minute period)",  
        "Rest for 3 hours (20 minute period)",
        f"Charge at 0.5C until {V_max} V (6 minute period)",
        f"Hold at {V_max}V until C/100",
        # Update 23-11-17: add one more 0.5C cycle to increase throughput capacity
        f"Discharge at 0.5C until {V_min} V (6 minute period)",  
        "Rest for 3 hours (20 minute period)",
        f"Charge at 0.5C until {V_max} V (6 minute period)",
        f"Hold at {V_max}V until C/100",
        "Rest for 3 hours (20 minute period)", 
        # refill and adjust to target SOC!!!
        #f"Charge at 0.3C until {V_max}V",
        #f"Hold at {V_max}V until C/100",
        #"Rest for 1 hours (20 minute period)", 
        # adjust to target SOC, SOC before this step must be 100%
        f"Discharge at 1C for {discharge_time_mins} minutes or until {V_min}V", # discharge for 15%SOC
        "Rest for 3 hours (20 minute period)", 
        ),  ])  


In [9]:
Sim_wocha_at_100 = pb.Simulation(
    Model_new,
    experiment = Exp_wocha_at_100, 
    parameter_values=Para_update_RPT, 
    solver = pb.CasadiSolver(),
    var_pts = var_pts,
    submesh_types=submesh_types
)
Sol_wocha_at_100 = Sim_wocha_at_100.solve(
    calc_esoh=False,
    # save_at_cycles = Update_Cycles,
    callbacks=Call_RPT) 

In [10]:
Sim_cha_at_100 = pb.Simulation(
    Model_new,
    experiment = Exp_cha_at_100, 
    parameter_values=Para_update_RPT, 
    solver = pb.CasadiSolver(),
    var_pts = var_pts,
    submesh_types=submesh_types
)
Sol_cha_at_100 = Sim_cha_at_100.solve(
    calc_esoh=False,
    # save_at_cycles = Update_Cycles,
    callbacks=Call_RPT) 

2023-12-11 00:48:53.456 - [ERROR] callbacks.on_experiment_error(224): Simulation error: Could not find consistent states: Could not find acceptable solution: Error in Function::call for 'roots' [Newton] at .../casadi/core/function.cpp:330:
.../casadi/core/rootfinder.cpp:277: rootfinder process failed. Set 'error_on_fail' option to false to ignore this error.


In [ ]:
label = ["0",] 
vars = [
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode potential [V]",
    "Positive current collector potential [V]", 
    "Terminal voltage [V]", 
    "Negative electrode porosity",
]
quick_plot = pb.QuickPlot(
    [Sol_new,], vars,label,variable_limits='fixed',
    time_unit='hours',n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()